In [ ]:
import os

from edf.pc_utils import draw_geometry, voxel_filter
from edf.data import PointCloud, SE3, TargetPoseDemo, DemoSequence, DemoSeqDataset
from edf.preprocess import Rescale, NormalizeColor, Downsample
from edf.agent import PickAgent

import numpy as np
import yaml
import plotly as pl
import plotly.express as ple
import open3d as o3d

import torch
from torch.utils.data import DataLoader
from torchvision.transforms import Compose

In [ ]:
device = 'cuda:0'
# device = 'cpu'
unit_len = 0.01

pick_agent_config_dir = "config/agent_config/pick_agent.yaml"
pick_agent_param_dir = "checkpoint/mug_10_demo/pick/model_iter_600.pt"
max_N_query_pick = 1
langevin_dt_pick = 0.001


pick_agent = PickAgent(config_dir=pick_agent_config_dir, 
                       device = device,
                       max_N_query = max_N_query_pick, 
                       langevin_dt = langevin_dt_pick)
pick_agent.load(pick_agent_param_dir)

load_transforms = Compose([Rescale(rescale_factor=1/unit_len),
                          ])
transforms = Compose([Downsample(voxel_size=1.0, coord_reduction="average"),
                      NormalizeColor(color_mean = torch.tensor([0.5, 0.5, 0.5]), color_std = torch.tensor([0.5, 0.5, 0.5])),
                     ])
trainset = DemoSeqDataset(dataset_dir="demo/test_demo", annotation_file="data.yaml", load_transforms = load_transforms, transforms=transforms, device=device)
# train_dataloader = DataLoader(trainset, shuffle=False, collate_fn=lambda xs:{'processed': [x['processed'] for x in xs], 'raw': [x['raw'] for x in xs]}) 
train_dataloader = DataLoader(trainset, shuffle=False, collate_fn=lambda x:x)

In [ ]:
for train_batch in train_dataloader:
    for data in train_batch:
        demo_seq_raw: DemoSequence = data['raw']
        demo_seq: DemoSequence = data['processed']
        break
    break

In [ ]:
T_seed = 100
pick_policy = 'sorted'
pick_mh_iter = 1000
pick_langevin_iter = 300
pick_dist_temp = 1.
pick_policy_temp = 1.
pick_optim_iter = 100
pick_optim_lr = 0.005

In [ ]:
Ts, edf_outputs, logs = pick_agent.forward(pc=demo.pick_demo.scene_pc, T_seed=T_seed, policy = pick_policy, mh_iter=pick_mh_iter, langevin_iter=pick_langevin_iter, 
                                            temperature=pick_dist_temp, policy_temperature=pick_policy_temp, optim_iter=pick_optim_iter, optim_lr=pick_optim_lr)

In [ ]:
draw_geometry(demo.pick_demo.scene_pc.to_pcd())